In [209]:
import pandas as pd
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
import ast
import numpy as np
pd.set_option('display.max_rows', None)

In [210]:
def deduct_payback(transact,df):
    amount = transact.amount
    amount_payback = 0
    for idx in transact.pb_assign:       
        if idx > 0:
            amount_payback = df[df.id == idx].amount.squeeze()
            if amount_payback <= - amount:
                amount += amount_payback
                amount_payback = 0
            else:
                amount_payback += amount
                amount = 0
        elif idx == 0:
            amount = 0
    transact.amount = amount
    transact["payback_rest"] = amount_payback
    return transact


In [258]:
def initialize_df(path):
    """ tbd """
    con = sqlite3.connect(path)
    df = pd.read_sql_query("SELECT * from transacts", con)
    df["date"] = pd.to_datetime(df["date"], unit='s',utc=True)
    df["date"] = df["date"].dt.tz_convert('Europe/Berlin')
    df["month-year"] = df["date"].apply(lambda date: "{} {}".format(date.month_name(),date.year))
    df["year"] = df["date"].apply(lambda date: "{}".format(date.year))
    df["pb_assign"] = df["pb_assign"].apply(lambda pb_assign: ast.literal_eval(pb_assign))
    df["payback_rest"] = 0
    #df[df["pb_assign"].str.len()>0] = df[df["pb_assign"].str.len()>0].apply(lambda transact: deduct_payback(transact,df),axis=1)
    return df

In [274]:
df = initialize_df("/Users/loris/Documents/MoneyManager/db_private.db")
df["year"] = df["date"].apply(lambda date: "{}".format(date.year))
#df = distribute_payback_rest(df)

In [214]:
def deduct_payback_rest(transact,payback_rest):
    if payback_rest > 0:
        print(payback_rest)
        if payback_rest < - transact.amount:
            transact.amount += payback_rest
            payback_rest = 0
        else:
            payback_rest += transact.amount
            transact.amount = 0
            
        transact.payback_rest = payback_rest
    return transact,payback_rest
    
def distribute_payback_rest(df):
    df_payback_rest = df[(df["payback_rest"]>0)]
    for id_pb,transact_pb in df_payback_rest.iterrows():
        df_same_tag = df[(df.tag == transact_pb.tag) & (df.sub_tag == transact_pb.sub_tag) & (df.id != transact_pb.id)]
        df_same_tag = df_same_tag.iloc[(df_same_tag.date-transact_pb.date).abs().argsort()]
        payback_rest = transact_pb.payback_rest
        for id_iter,transact_iter in df_same_tag.iterrows():
            if payback_rest > 0:
                if payback_rest < - transact_iter.amount:
                    df.at[id_iter, 'amount'] += payback_rest
                    payback_rest = 0
                    break
                else:
                    payback_rest += transact_iter.amount
                    df.at[id_iter, 'amount'] = 0
                
            df.at[id_iter, 'payback_rest'] = payback_rest

    return df
    

In [215]:
def adjust_amounts(df):
    # Sort by tag and amount (descending for positive amounts, ascending for negative)
    df_payback_rest = df[df["payback_rest"]>0]
    df_payback_rest = df_payback_rest.sort_values(by=['tag', 'amount'], ascending=[True, True])
    
    # Iterate over unique tags
    for id,transact in df_payback_rest.iterrows():
        tag = transact.tag
        print(tag)
        pos_indices = df[(df['tag'] == tag) & (df['amount'] > 0)].index
        neg_indices = df[(df['tag'] == tag) & (df['amount'] < 0)].index
        
        for pos_idx in pos_indices:
            pos_val = df.at[pos_idx, 'amount']
            
            for neg_idx in neg_indices:
                neg_val = df.at[neg_idx, 'amount']
                
                if neg_val == 0:
                    continue
                
                if pos_val + neg_val >= 0:
                    df.at[pos_idx, 'amount'] = pos_val + neg_val
                    df.at[neg_idx, 'amount'] = 0
                    break
                else:
                    df.at[neg_idx, 'amount'] = 0
                    pos_val += neg_val
                    df.at[pos_idx, 'amount'] = pos_val

    return df

In [216]:
selected_month_year = "2020"
year_transacts = df[df["year"] == selected_month_year]
going_out = year_transacts[(year_transacts["amount"]<0)]["amount"].sum()
going_in = year_transacts[(year_transacts["amount"]>0)]["amount"].sum()


In [217]:
def check_payback_year(transact,df):
    amount = transact.amount
    amount_payback = 0
    for idx in transact.pb_assign:
        if idx > 0:       
            #if df[df.id == idx].squeeze().year != transact.year:
            #print(df[df.id == idx].squeeze().year, transact.year)
            if df[df.id == idx].squeeze().year != transact.year:
                print("stop")
    #return transact

In [218]:
paybacks_list = year_transacts[year_transacts['pb_assign'].apply(lambda x: x == [0])].id.values
for idx in paybacks_list:
    if df[df["pb_assign"].apply(lambda x: idx in x)].squeeze().year != df[df["id"]==idx].squeeze().year:
        print(df[df["id"]==idx].squeeze())
    

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
import json
accounts_file = mm_dir_path / "accounts.json"
accounts = importAllAccounts(accounts_file)
with open(accounts_file, "r", encoding="utf-8") as json_file:
    account_data = json.load(json_file)

if "accounts" in account_data:
    for acc in account_data["accounts"]:
        signs = (acc["signs"] if "signs" in acc else None)
        account = Account(acc["name"], acc["balance"],acc["rowsDeleted"], acc["colsDeleted"], acc["headers"],acc["detectString"],acc["dmy_format"],acc["ignoreTypes"],signs=signs,balance_base=acc["balance_base"])
        accounts.append(account)


In [286]:
import json
import plotly.graph_objects as go
accounts_file = "/Users/loris/src/Python/MoneyManager/accounts.json"
with open(accounts_file, "r", encoding="utf-8") as json_file:
    account_data = json.load(json_file)
account_data = account_data["accounts"]

In [275]:
def cumsum_with_start(group, start_value):
    return group.cumsum() + start_value
starting_balance = {account["name"]:list(account["balance_base"].values())[0] for account in account_data}
df = df.sort_values("date")
df['account_balance'] = df.groupby('account')['amount'].apply(lambda group: cumsum_with_start(group, starting_balance[group.name]))

-3.25
-400.0
9.62
15.0
-70.0
-4.99


In [292]:
fig = go.Figure()
for account, group in df.groupby('account'):
    fig.add_trace(go.Scatter(
        x=group.date,
        y=group['account_balance'],
        mode='lines',
        line_shape='hv',  # horizontal-vertical step line
        name=f'{account}',
        marker=dict(size=10)
    ))

fig.update_layout(
    xaxis_title='Zeit',
    yaxis_title='Kontostand',
    legend_title='Konto',
    template='simple_white'
)

fig.show()

In [235]:
grouped= df.groupby('account').count()
for account, group in grouped:
    print(f"Account: {account}")
    print(group)

Account: Compte courant
        id                      date                           type  \
0       35 2018-10-17 00:00:00+02:00     UTILISATION CARTE DE DEBIT   
1       36 2018-10-17 00:00:00+02:00     UTILISATION CARTE DE DEBIT   
2       37 2018-10-17 00:00:00+02:00     UTILISATION CARTE DE DEBIT   
3       38 2018-10-17 00:00:00+02:00     UTILISATION CARTE DE DEBIT   
4       39 2018-10-18 00:00:00+02:00             DOMICILIATION SEPA   
5       40 2018-10-19 00:00:00+02:00     UTILISATION CARTE DE DEBIT   
6       41 2018-10-23 00:00:00+02:00     UTILISATION CARTE DE DEBIT   
7       42 2018-10-26 00:00:00+02:00     UTILISATION CARTE DE DEBIT   
8       43 2018-10-26 00:00:00+02:00     UTILISATION CARTE DE DEBIT   
9       44 2018-10-31 00:00:00+01:00     UTILISATION CARTE DE DEBIT   
10      45 2018-10-31 00:00:00+01:00     UTILISATION CARTE DE DEBIT   
11      46 2018-11-02 00:00:00+01:00                   BONIFICATION   
12      47 2018-11-02 00:00:00+01:00     UTILISATION 